# FizzBuzz in Qiskit

**Fizz buzz** is a group word game for children to teach them about division. Someone chosen by lot starts counting from $1$ then players take turns incerementing a previous number and saying the result except on occasion it should be replaced by
* *Fizz* if the result is divisible by $3$,
* *Buzz* if the result is divisible by $5$,
* *Fizz Buzz* if both occur simultaneously.

A wrong answer means the player is out of a game continued round by round. It would eventually a word chain likes "One, Two, *Fizz*, Four, *Buzz*, *Fizz*, Seven, ..." and a winner who never made a mistake.

**FizzBuzz** is also a common programming task to test basic skills. In this case a sequence from $1$ to $100$ should be a program output and could be obtained in many ways. That is why there are a lot of its implementation in different programming languages. Here we realize a simplified version of FizzBuzz in Qiskit using Grover's Search Algorithm (GSA) for multiple solutions.

Classical approaches are generally based on for loop which body contains a divisibility test. A number of iterations $N$ then equals to length of input so time complexity is a linear function $O\left(N\right)$. Actually it may vary slightly because a database is not a complitely unstructured meanwhile quantum computing looks promising to improve significantly.

GSA is an unstructured search algorithm presented by Lov Grover in 1996. Instead of brut force it processes all data in parallel increasing a probability of favorable outcomes in final distribution. To find $k$ solutions in array of $N$ elements we perform $\lfloor\frac\pi4\sqrt\frac N K\rfloor$ iterations thus time complexity now is a square root function $O\left(\sqrt N\right)$. This is quadratic speedup that is why GSA provides quantum supremacy.

The goal is to create a sequence from $0$ to $15$ due to some reasons. All the numbers have a four bit representation, two auxiliary bits are enough to carry a result of check condition that is simplified in the range, and there are two *FizzBuzz* opening and closing the chain.

We break down the task into *Fizz* and *Buzz* parts devising two divisibility tests independent of one another. However their results are very similar. To avoid any confusion we build two quantum circuits differ in the test but have an oracle and a diffuser the same. At the end all measurement results will be processed together to create an output. There are six multiples of $3$ which are $\{ 0, 3, 6, 9, 12, 15\}$ and four multiples of $5$ which are $\{ 0, 5, 10, 15\}$ thus the oracle is applied $\lfloor\frac\pi4\sqrt\frac{16}6\rfloor=1$ time in *Fizz* and $\lfloor\frac\pi4\sqrt\frac{16}4\rfloor=1$ time in *Buzz* circuits. On the whole we call the oracle twice.

The total number of iterations doesn't depend on input length. Each third number is divisible by $3$ so $\frac Nk\approx3$ and each fifth number is divisible by $5$ so $\frac Nk\approx5$. We should make $\lfloor\frac\pi4\sqrt3\rfloor=1$ *Fizz* and $\lfloor\frac\pi4\sqrt5\rfloor=1$ *Buzz* iterations. It means constant time complexity $O\left(2\right)$ conditioned by some input structure.

Given a binary string $$c_5c_4c_3c_2c_1c_0$$ where two most significant bits $c_5c_4$ keep some information about a number encoded in four least significant bits $c_3c_2c_1c_0$. To acquire the string or rather a string list we measure in $Z$ basis some quantum register $$\lvert q_5q_4q_3q_2q_1q_0\rangle.$$

Any computation is a quantum circuit, i.e. a sequence of unitaries apply on qubits for state evolution then measurements collapse a state to a string. Because the final part is probabilistic we run a  circuit on real device or simulator many times in a row. A result is a dictionary with binary strings are the keys and count outcomes are the values. Here it is required only 6 qubits not split into main and ancilla registers in a code but they can be mentioned this way in description.

## Imports

All required functions such as Grover Oracle, Quantum Fourier Transform, etc., are defined below and it is enough to import some standard Qiskit modules and last but not least numpy to get the $\pi$ value.

In [ ]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
import numpy as np

## Initialization

At the beginning we apply Hadamard gate on whole main register creating a uniform superposition over all states. Now we are allowed to test the numbers in parallel.

In [ ]:
def init(qc):
    qc.h(range(4))
    qc.barrier()

## Quantum Fourier Transform

Quantum Fourier Transform (QFT) is a number one subroutine in quantum computing. This way we perform a change of basis for some state turning its representation from bit to phase. Note that QFT defined here switches a qubit ordering so that MSB becomes LSB and vice versa. It can be fixed using extra $SWAP$ gates but we just reckon with it in computations.

In [ ]:
def qft(qc):
    qc.h(5)
    qc.cp(np.pi/2, 4, 5)
    qc.h(4)
    qc.barrier()

## Divisibility by 3

We have $3_{10}=11_2$ and it is very easy to check if a binary number satisfies the condition. Same as for $11$ in decimal the algorithm takes an alternating digit sum. In other word we find the sums of its even and odd digits and compare them to each other. If they are equal or their difference is multiple of $11_2$ then the binary number is divisible by $11_2$. A maximum absolute difference in a four bit register equals to $10_2$ thus the second statement is always false and the divisibility test reduces to $$c_3-c_2+c_1-c_0\equiv0.$$

It is not applicable for large registers where a non-zero alternating digit sum may occur for multiples of $3$ too. Then either an oracle should consider that or the sum itself should be tested the same way again. The latter is much easier but it requires additional qubits for intermediate calculations.

The left-hand side can be done in two ways: a bit adder with $CX$ and $CCX$ gates or a phase adder with $CP$ gates. We choose the second one combines addition and substraction as an algebraic sum. It performs counterclockwise and clockwise phase rotations of ancilla qubits depend on sign.

In [ ]:
def div_by_3(qc):
    qc.cp(np.pi/2, 3, 5)
    qc.cz(3, 4)
    qc.cp(-np.pi/2, 2, 5)
    qc.cz(2, 4)
    qc.cp(np.pi/2, 1, 5)
    qc.cz(1, 4)
    qc.cp(-np.pi/2, 0, 5)
    qc.cz(0, 4)
    qc.barrier()

For amplitude amplification to work correct it is required to set ancilla register to $\lvert00\rangle$ state before diffusion. The stage is called 'uncomputation'. That is why we create a function inverse to above one. In phase addition it means all rotations just reverse their directions. Note that $CP(\pi)=CP(-\pi)=CZ$ so the gate remains unchanged.

In [ ]:
def inv_div_by_3(qc):
    qc.cp(-np.pi/2, 3, 5)
    qc.cz(3, 4)
    qc.cp(np.pi/2, 2, 5)
    qc.cz(2, 4)
    qc.cp(-np.pi/2, 1, 5)
    qc.cz(1, 4)
    qc.cp(np.pi/2, 0, 5)
    qc.cz(0, 4)
    qc.barrier()

## Divisibility by 5

It seems a bit harder but very similar to the previous one. Now we have $3_{10}=11_4={0101}_2$ and each quaternary digit turns to a binary pair thus we compare the sums of even and odd pairs of digits. Just like before if they are equal or their difference is multiple of $0101_2$ then the binary number is divisible by $0101_2$. In four bit register there are only two pairs, the odd and the even, so we go straight to substraction skipping any addition. In this case a maximum absolute value of the difference could equal to $11_2$. The second statement is always false again and the divisibility test reduces to $$c_3c_2-c_1c_0\equiv0.$$

In [ ]:
def div_by_5(qc):
    qc.cp(np.pi/2, 2, 5)
    qc.cz(2, 4)
    qc.cz(3, 5)
    qc.cp(-np.pi/2, 0, 5)
    qc.cz(0, 4)
    qc.cz(1, 5)
    qc.barrier()

Now we create an inverse function for uncomputation changing all rotations to opposite ones.

In [ ]:
def inv_div_by_5(qc):
    qc.cp(-np.pi/2, 2, 5)
    qc.cz(2, 4)
    qc.cz(3, 5)
    qc.cp(np.pi/2, 0, 5)
    qc.cz(0, 4)
    qc.cz(1, 5)
    qc.barrier()

## Inverse Quantum Fourier Transform

After arithmetic calculations we return from phase to bit representation of ancilla register. Superposition of states differ phases converts here to a separable state. We get some tensor product of Pauli $Z$ eigenstates which are $\lvert0\rangle$ or $\lvert1\rangle$. Note the main register remains in superposition and we still process all numbers simultaneously. Now each of them encoded in four least significant qubits goes along with only one possible alternating digit sum encoded in two most significant qubits.

In [ ]:
def iqft(qc):
    qc.h(4)
    qc.cp(-np.pi/2, 4, 5)
    qc.h(5)
    qc.barrier()

## Oracle

Grover oracle marks the states we wish to find by flipping their relative phase to the negative one $$U_\omega\lvert\psi\rangle=\begin{cases}\hphantom{-}\lvert\psi\rangle\,\text{for the 'good' states},\\ -\lvert\psi\rangle\,\text{for the others}.\end{cases}$$
What does 'good' mean in our example? Each test defined above returns $\lvert00\rangle$ ancilla if and only if a number is divisible. The register is two qubits so we need an operator switches one state out of four. $CZ$ gate looks suitable for both quantum circuits but in fact it flips a $\lvert11\rangle$ phase. To use it here we additionally surround the operator by $XX$ gates.

In [ ]:
def oracle(qc):
    qc.x(range(4, 6))
    qc.cz(4, 5)
    qc.x(range(4, 6))
    qc.barrier()

## Diffuser

Grover diffusion operator $U_s=2\,\lvert0^{\otimes4}\rangle\langle0^{\otimes4}\rvert-I$ makes another one reflection and unlike to above it switches almost all terms in superposition, not only the searched states. This way we amplify a probability amplitude for 'good' outcomes by reducing it for the others. Just like an oracle a diffuser is identical for both quantum circuits and actually the same for almost any search. But unlike an oracle in applies on the main register, not the ancilla.

In [ ]:
def diffuser(qc):
    qc.h(range(4))
    qc.x(range(4))
    qc.mcp(np.pi, [0, 1, 2], 3)
    qc.x(range(4))
    qc.h(range(4))
    qc.barrier()

## Fizz

Now we put all functions together and build up a quantum circuit to find multiples of $3$. From creating superposition through complex oracle and diffuser we finish by measurement in Pauli $Z$ basis.

In [ ]:
fizz = QuantumCircuit(6)
init(fizz)
qft(fizz)
div_by_3(fizz)
iqft(fizz)
oracle(fizz)
qft(fizz)
inv_div_by_3(fizz)
iqft(fizz)
diffuser(fizz)
fizz.measure_all()
fizz.draw()

Send the circuit to device for executing. Here we get a main simulator of the Aer provider as a backend. It mimics a behavior of real quantum hardware systems those may not be available to everyone. By default the simulator is noise-free what is useful for education or debugging. We are also possible to run the circuits on real quantum devices however an error correction procedure may be required.

In [ ]:
simulator = Aer.get_backend('aer_simulator')
job_fizz = execute(fizz, simulator, shots = 1024)
result_fizz = job_fizz.result()
counts_fizz = result_fizz.get_counts()
plot_histogram(counts_fizz, figsize = (18, 6))

## Buzz

Exactly the same way we create and then execute another circuit to find multiples of $5$.

In [ ]:
buzz = QuantumCircuit(6)
init(buzz)
qft(buzz)
div_by_5(buzz)
iqft(buzz)
oracle(buzz)
qft(buzz)
inv_div_by_5(buzz)
iqft(buzz)
diffuser(buzz)
buzz.measure_all()
buzz.draw()

In [ ]:
job_buzz = execute(buzz, simulator, shots = 1024)
result_buzz = job_buzz.result()
counts_buzz = result_buzz.get_counts()
plot_histogram(counts_buzz, figsize = (18, 6))

## Ouptut

After both circuits running we have obtained two dictionaries plotted above as histograms. Each high bar corresponds to some binary string with required number in four LSBs (main register). This is an expected result because the strings describe good states amplified by GSA. They are obviously distinguish from others but it is still raw data.

The output we work on is a list of strings so create some pattern for the beginning. Then apply firstly 'Fizz' and secondly 'Buzz' results to elements those indices are decoded from amplified strings main register. Other strings are still empty should be filled out their indices directly.

In [ ]:
output = [''] * 16
for key in {k:v for (k,v) in counts_fizz.items() if v>100}:
    output[int(key[2:], 2)] += 'Fizz'
for key in {k:v for (k,v) in counts_buzz.items() if v>100}:
    output[int(key[2:], 2)] += 'Buzz'
#for out in output if out == '':
#    out += '1'
#[i += str(index(i)) for i in output]
#[print(i, end=' ') for i in output]
print(output)